In [1]:
from sqlalchemy import create_engine, Column, String, Boolean, Integer, Float, Numeric, MetaData, Table, type_coerce, ForeignKey
from sqlalchemy.orm import mapper, create_session, relationship
from sqlalchemy.ext.declarative import declarative_base
import sqlalchemy.types as types
import re
import os
import glob
import numpy as np

In [19]:
basepath='/ocean/shared/SalishSeaCastData/DFO/CTD/'
basedir=basepath + 'req20181116/'
dbname='DFO_CTD'
# if more paths added later (additional data requests) see createDBfromDFO_OPDB.py for how to add
fout=open(basepath+'createDBfromDFO_CTD_log.txt','w')
ferr=open(basepath+'createDBfromDFO_CTD_errors.txt','w')
fout.write('Files processed:\n')

dirs0=[os.path.join(basepath,x) for x in os.listdir(basepath) if (os.path.isdir(basepath+x) and not re.match('^\.', x))]
dirs1=list()

In [20]:
dirs0

['/ocean/shared/SalishSeaCastData/DFO/CTD/req20200928',
 '/ocean/shared/SalishSeaCastData/DFO/CTD/req20181116']

In [21]:
    for ireq in dirs0:
        dirs1=dirs1+[os.path.join(ireq,x) for x in os.listdir(ireq) \
                     if (os.path.isdir(os.path.join(ireq,x)) and not re.match('^\.', x))]
    dirs1.sort()

In [22]:
dirs1

['/ocean/shared/SalishSeaCastData/DFO/CTD/req20181116/EO UBC November 16, 2018 (2014 data)',
 '/ocean/shared/SalishSeaCastData/DFO/CTD/req20181116/EO UBC November 16, 2018 (2015 data)',
 '/ocean/shared/SalishSeaCastData/DFO/CTD/req20181116/EO UBC November 16, 2018 (2016 data a)',
 '/ocean/shared/SalishSeaCastData/DFO/CTD/req20181116/EO UBC November 16, 2018 (2016 data b)',
 '/ocean/shared/SalishSeaCastData/DFO/CTD/req20181116/EO UBC November 16, 2018 (2017 data)',
 '/ocean/shared/SalishSeaCastData/DFO/CTD/req20181116/EO UBC November 16, 2018 (2018 data)',
 '/ocean/shared/SalishSeaCastData/DFO/CTD/req20200928/CTD1',
 '/ocean/shared/SalishSeaCastData/DFO/CTD/req20200928/CTD2']

In [24]:
filenames1=list()
bnamesAll=list() 
for cdirpath in dirs1:
    filenames1=filenames1+[os.path.join(cdirpath,f) for f in os.listdir(cdirpath) if ((f not in bnamesAll) and (not re.match('.*jpg$',f)))]
    bnamesAll=bnamesAll+[f for f in os.listdir(cdirpath)]
    # left over from nutrients version where multiple requests led to overlap; retain for future use
filenames1.sort()
filenames=filenames1 #contains full paths

In [26]:
filenames[-1]

'/ocean/shared/SalishSeaCastData/DFO/CTD/req20200928/CTD2/lbb_20190123_20190810_0336m_L2.ctd'

In [31]:
testline='    #       <PathLength>0.250</PathLength>'

In [41]:
f=re.search('(?<=PathLength>).*(?=</PathLength)',testline)

In [42]:
f

<re.Match object; span=(24, 29), match='0.250'>

In [43]:
f[0]

'0.250'

In [44]:
if f:
    y=f[0]
y

'0.250'

In [49]:
testline=' #     <!-- A/D voltage 2, Transmissometer, WET Labs C-Star -->'

In [50]:
f=re.search('Transmissometer,',testline)

In [51]:
f

<re.Match object; span=(27, 43), match='Transmissometer,'>

In [52]:
XmissPatterns={'xmiss_Serial':re.compile('(?<=SerialNumber>).*(?=</SerialNumber)'),
               'xmiss_CalDate':re.compile('(?<=CalibrationDate>).*(?=</CalibrationDate)'),
               'xmiss_m':re.compile('(?<=M>).*(?=</M)'),
               'xmiss_b':re.compile('(?<=B>).*(?=</B)'),
               'xmiss_PathLen':re.compile('(?<=PathLength>).*(?=</PathLength)')}

In [53]:
lines=('    #     <!-- A/D voltage 2, Transmissometer, WET Labs C-Star -->',
'    #     <WET_LabsCStar SensorID="71" >',
'    #       <SerialNumber>953DR</SerialNumber>',
'    #       <CalibrationDate>Aug 9th 2017</CalibrationDate>',
'    #       <M>19.2982</M>',
'    #       <B>-1.1000</B>',
'    #       <PathLength>0.250</PathLength>',
'    #     </WET_LabsCStar>')

In [56]:
for line in lines:
    for key, pattern in XmissPatterns.items():
        print(line)
        match=pattern.search(line)
        if match is None:
            continue
        print(key,match[0])
        break

    #     <!-- A/D voltage 2, Transmissometer, WET Labs C-Star -->
    #     <!-- A/D voltage 2, Transmissometer, WET Labs C-Star -->
    #     <!-- A/D voltage 2, Transmissometer, WET Labs C-Star -->
    #     <!-- A/D voltage 2, Transmissometer, WET Labs C-Star -->
    #     <!-- A/D voltage 2, Transmissometer, WET Labs C-Star -->
    #     <WET_LabsCStar SensorID="71" >
    #     <WET_LabsCStar SensorID="71" >
    #     <WET_LabsCStar SensorID="71" >
    #     <WET_LabsCStar SensorID="71" >
    #     <WET_LabsCStar SensorID="71" >
    #       <SerialNumber>953DR</SerialNumber>
xmiss_Serial 953DR
    #       <CalibrationDate>Aug 9th 2017</CalibrationDate>
    #       <CalibrationDate>Aug 9th 2017</CalibrationDate>
xmiss_CalDate Aug 9th 2017
    #       <M>19.2982</M>
    #       <M>19.2982</M>
    #       <M>19.2982</M>
xmiss_m 19.2982
    #       <B>-1.1000</B>
    #       <B>-1.1000</B>
    #       <B>-1.1000</B>
    #       <B>-1.1000</B>
xmiss_b -1.1000
    #       <PathLength>0.

In [59]:
for line in lines:
    for key, pattern in XmissPatterns.items():
        print(line)
        match=pattern.search(line)
        if match is not None:
            print(key,match[0])
            break

    #     <!-- A/D voltage 2, Transmissometer, WET Labs C-Star -->
    #     <!-- A/D voltage 2, Transmissometer, WET Labs C-Star -->
    #     <!-- A/D voltage 2, Transmissometer, WET Labs C-Star -->
    #     <!-- A/D voltage 2, Transmissometer, WET Labs C-Star -->
    #     <!-- A/D voltage 2, Transmissometer, WET Labs C-Star -->
    #     <WET_LabsCStar SensorID="71" >
    #     <WET_LabsCStar SensorID="71" >
    #     <WET_LabsCStar SensorID="71" >
    #     <WET_LabsCStar SensorID="71" >
    #     <WET_LabsCStar SensorID="71" >
    #       <SerialNumber>953DR</SerialNumber>
xmiss_Serial 953DR
    #       <CalibrationDate>Aug 9th 2017</CalibrationDate>
    #       <CalibrationDate>Aug 9th 2017</CalibrationDate>
xmiss_CalDate Aug 9th 2017
    #       <M>19.2982</M>
    #       <M>19.2982</M>
    #       <M>19.2982</M>
xmiss_m 19.2982
    #       <B>-1.1000</B>
    #       <B>-1.1000</B>
    #       <B>-1.1000</B>
    #       <B>-1.1000</B>
xmiss_b -1.1000
    #       <PathLength>0.

In [1]:


import sqlalchemy
from sqlalchemy import create_engine, Column, String, Boolean, Integer, Float, MetaData, Table, type_coerce, ForeignKey, case, update
from sqlalchemy.orm import mapper, create_session, relationship, aliased, Session
from sqlalchemy.ext.declarative import declarative_base
import csv
from sqlalchemy import case
import numpy as np
from sqlalchemy.ext.automap import automap_base
import matplotlib.pyplot as plt
import sqlalchemy.types as types
import numbers
from sqlalchemy.sql import and_, or_, not_, func
from sqlalchemy.sql import select
import os
import glob
import re
from os.path import isfile
import gsw
import pandas as pd
from sqlalchemy.sql import and_, or_, not_, func


In [2]:
# definitions
basedir='/ocean/shared/SalishSeaCastData/DFO/CTD/'
dbname='DFO_CTD'
# if db does not exist, exit
# 1st, set Include=False for all CastAway data and duplicates
engine0 = create_engine('sqlite:///' + basedir + dbname + '.sqlite', echo = False)
md=MetaData()
md.reflect(engine0)
Base0 = automap_base(metadata=md)
# reflect the tables in salish.sqlite:
Base0.prepare()
# mapped classes have been created
# existing tables:
StationTBL0=Base0.classes.StationTBL
AncillaryTBL0=Base0.classes.AncillaryTBL
ObsTBL0=Base0.classes.ObsTBL
session0 = create_session(bind = engine0, autocommit = False, autoflush = True)

In [9]:
temp=pd.DataFrame(session0.query(StationTBL0.ID,StationTBL0.Lat,StationTBL0.Lon,StationTBL0.Include,ObsTBL0.Pressure,
               ObsTBL0.Salinity,ObsTBL0.Salinity_T0_C0,ObsTBL0.Salinity_T1_C1,
               ObsTBL0.Temperature_Primary,ObsTBL0.Temperature_Secondary,ObsTBL0.Include,ObsTBL0.PAR,ObsTBL0.PAR1,
                                ObsTBL0.sourceFile).select_from(StationTBL0).\
                   join(ObsTBL0,ObsTBL0.StationTBLID==StationTBL0.ID).\
                filter(StationTBL0.ID==120).all())

In [14]:
test=pd.DataFrame(session0.query(StationTBL0.ID,StationTBL0.Lat,StationTBL0.Lon,StationTBL0.Include).\
            filter(StationTBL0.ID==120).all())

In [15]:
test

,ID,Lat,Lon,Include
0,120,48.056333,-125.599167,True


In [ ]:
test=pd.DataFrame(session0.query(StationTBL0.ID,StationTBL0.Lat,StationTBL0.Lon,StationTBL0.Include).\
            filter(StationTBL0.ID==120).all())

In [ ]:
test

In [10]:
temp

""


In [11]:
temp2=pd.DataFrame(session0.query(StationTBL0.ID,StationTBL0.Lat,StationTBL0.Lon,StationTBL0.Include,ObsTBL0.Pressure,
               ObsTBL0.Salinity,ObsTBL0.Salinity_T0_C0,ObsTBL0.Salinity_T1_C1,
               ObsTBL0.Temperature_Primary,ObsTBL0.Temperature_Secondary,ObsTBL0.Include,ObsTBL0.PAR,ObsTBL0.PAR1,
                                ObsTBL0.sourceFile).select_from(StationTBL0).\
                   join(ObsTBL0,ObsTBL0.StationTBLID==StationTBL0.ID).\
                filter(StationTBL0.ID==135).all())

In [12]:
temp2

""


In [13]:
for el in np.unique(temp['sourceFile']):
    print(el)

KeyError: 'sourceFile'

In [ ]:
len(temp)